In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Crie uma frase sobre: {assunto}")

In [ ]:
chain = prompt | model

In [ ]:
chain

In [ ]:
chain.invoke({"assunto": "Python"})

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [ ]:
chain.invoke({"assunto": "Python"})

In [ ]:
input = {"assunto":"Python"}

In [ ]:
prompt_format = prompt.invoke(input)
prompt_format

In [ ]:
resposta = model.invoke(prompt_format)
resposta

In [ ]:
from langchain.chains.llm import LLMChain
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-3.5-turbo")

prompt = ChatPromptTemplate.from_template("Crie uma frase sobre o assunto {assunto}")

output_parser = StrOutputParser()

In [ ]:
chain = LLMChain(
    llm=model,
    prompt=prompt,
    output_parser=output_parser
)

chain.invoke({"assunto": "Python"})

In [ ]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
caminhos = [
    "files/LLM.pdf",
    ]

paginas = []

for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())

recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(paginas)

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [ ]:
from langchain_community.vectorstores.faiss import FAISS

vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embeddings_model
)
retriever = vectorstore.as_retriever(search_type='mmr')

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI()

template_str = '''Responda as perguntas do usuário com base no contexto fornecido.

Contexto: {contexto}

Pergunta: {pergunta}'''

template = ChatPromptTemplate.from_template(template_str)
output_parser = StrOutputParser()

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup_retrieval = RunnableParallel(
    {"pergunta": RunnablePassthrough(),
     "contexto": retriever}
)

chain = setup_retrieval | template | model | output_parser

In [ ]:
chain.invoke("O que é LLM?")

In [ ]:
retriever.invoke('O que é o LLM?')